In [22]:
from agents import *
from random import choice

In [20]:
class CheeseMaze(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location)
        
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            print("The rat hit itself with an edge")
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {} wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Cheese)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "hit":
            items = self.list_things_at(agent.location, tclass=Wall)
            if len(items) != 0:
                if agent.hit(items[0]):
                    print('{} hit itself with {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    
    def is_done(self):
        no_edibles = not any(isinstance(thing, Cheese) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

In [38]:
class TestRat(Agent):
    next_step = 1
    next_guide = 0
    location = [0,1]
    direction = Direction("right")
    
    def moveforward(self, success=True):
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.next_step = 1
            self.next_guide = 0
            self.location[self.next_guide] += self.next_step
        elif self.direction.direction == Direction.L:
            self.next_step = -1
            self.next_guide = 0
            self.location[self.next_guide] += self.next_step
        elif self.direction.direction == Direction.D:
            self.next_step = 1
            self.next_guide = 1
            self.location[self.next_guide] += self.next_step
        elif self.direction.direction == Direction.U:
            self.next_step = -1
            self.next_guide = 1
            self.location[self.next_guide] += self.next_step
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        if isinstance(thing, Cheese):
            return True
        return False

    def hit(self, thing):
        if isinstance(thing, Wall):
            self.next_step *= -1
            self.location[self.next_guide] += self.next_step
            return True
        return False

In [16]:
class Cheese(Thing):
    pass

In [18]:
class Wall(Thing):
    pass

In [19]:
def program(percepts):
    for perceptive in percepts:
        if isinstance(perceptive, Cheese):
            return 'eat'
        if isinstance(perceptive, Wall):
            return 'hit'
            turn = False
            choice = random.choice((1, 2, 3, 4))
        if isinstance(perceptive, Bump):
            turn = False
            choice = random.choice((1, 2))
        else:
            choice = random.choice((1, 2, 3, 4))
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

In [39]:
test_field = CheeseMaze(5,5, color={'TestRat': (200,0,0), 'Cheese': (230, 115, 40), 'Wall': (69, 92, 245)})

rat = TestRat(program)
test_field.add_thing(rat, [0,0])

rat_goal_1 = Cheese()
test_field.add_thing(rat_goal_1, [1,2])

rat_goal_2 = Cheese()
test_field.add_thing(rat_goal_2, [4,3])

maze_walls = [[0, 1], [1, 1], [2, 1],
              [2, 2], [1, 4], [4, 1],
              [4, 2], [4, 4], [0, 4]]

for position in maze_walls:
    wall = Wall()
    test_field.add_thing(wall, position)

print("Rat started at [0,0], facing right. Let's see if he found any cheese!")

test_field.run(50)

,,,,
,,,,
,,,,
,,,,
,,,,
